In [1]:
import math
from datetime import datetime as dt

import numpy as np
import pandas as pd
import planets
import xarray as xr
from tqdm.auto import tqdm

import heat1d

In [2]:
lat = 15

In [4]:
class ModelDatabase:
    default_chis = np.arange(1, 3.6, 0.1)

    def __init__(self, chis=None):
        if chis is None:
            self.chis = self.default_chis
        self.confs = [heat1d.Configurator(chi=chi) for chi in self.chis]

    def create_lat_models(self, lat):
        "For given latitude `lat`, create all chi-models."
        models = [
            heat1d.Model(
                planet=planets.Moon, lat=math.radians(lat), ndays=5, config=conf
            )
            for conf in self.confs
        ]
        self.current_lat = lat
        self.models = models

    def run_models(self):
        for model in tqdm(self.models, leave=False):
            model.run()

    def resample_and_xarray(self):
        "Resample to default time bin and store in xr.DataArray."
        mdf = pd.DataFrame()

        for m in self.models:
            mdf[m.chi] = m.resample_surface_cooling(binning="15min")

        self.mdf = mdf
        da = xr.DataArray(
            mdf.values,
            dims=["time", "chi"],
            coords={"time": mdf.index, "chi": self.chis},
            name="cooling_curves",
        )
        da = da.assign_coords(lat=self.current_lat)
        self.da = da

    def produce_one_lat(self, lat):
        "Produce models for all chis for one latitude."
        self.create_lat_models(lat)
        self.run_models()
        self.resample_and_xarray()
        return self.da

    def create_database(self, latitudes):
        arrays = []
        for lat in tqdm(latitudes):
            arrays.append(self.produce_one_lat(lat))
        stacked = xr.concat(arrays, "lat")
        stacked.name = "model_db"
        ds = stacked.to_dataset()
        now = dt.now().isoformat()[:16]
        savename = f"heat1d_model_database_{now}.zarr"
        ds.to_zarr(savename)
        self.ds = ds

In [17]:
modeler = ModelDatabase()

In [38]:
modeler.create_database(np.arange(0,81))

  0%|          | 0/81 [00:00<?, ?it/s]

  0%|          | 0/26 [00:00<?, ?it/s]

  0%|          | 0/26 [00:00<?, ?it/s]

  0%|          | 0/26 [00:00<?, ?it/s]

  0%|          | 0/26 [00:00<?, ?it/s]

...

  0%|          | 0/26 [00:00<?, ?it/s]

  0%|          | 0/26 [00:00<?, ?it/s]

  0%|          | 0/26 [00:00<?, ?it/s]

  0%|          | 0/26 [00:00<?, ?it/s]

  0%|          | 0/26 [00:00<?, ?it/s]

In [18]:
modeler.produce_one_lat(10)

  0%|          | 0/26 [00:00<?, ?it/s]

<xarray.DataArray 'cooling_curves' (time: 40, chi: 26)>
array([[107.27019071, 107.52806079, 107.77941487, ..., 112.20927326,
        112.36529454, 112.51900171],
       [106.01464512, 106.31574198, 106.50927776, ..., 110.97213079,
        111.15308145, 111.33078491],
       [104.89891859, 105.23425719, 105.3783303 , ..., 109.9178366 ,
        110.06683252, 110.26364671],
       ...,
       [ 89.81873108,  90.10610083,  90.4027737 , ...,  94.8786438 ,
         95.05146055,  95.21090981],
       [ 89.62643033,  89.90216095,  90.20378384, ...,  94.67701109,
         94.84276988,  95.00582356],
       [ 89.43890806,  89.71997861,  90.00998441, ...,  94.46994729,
         94.63924487,  94.80571352]])
Coordinates:
  * time     (time) datetime64[ns] 1970-01-01T07:37:30 ... 1970-01-01T17:22:30
  * chi      (chi) float64 1.0 1.1 1.2 1.3 1.4 1.5 ... 3.0 3.1 3.2 3.3 3.4 3.5
    lat      int64 10

In [19]:
model = modeler.models[0]

In [31]:
s = model.last_surface_cooling
s.index = heat1d.get_datetimeindex(s)

In [34]:
g1 = s.hvplot()
g1

:Curve   [index]   (0)

In [35]:
g2 = model.resample_surface_cooling().hvplot.scatter()
g2

:Scatter   [index]   (0)

In [36]:
g1 * g2

:Overlay
   .Curve.A_0   :Curve   [index]   (0)
   .Scatter.A_0 :Scatter   [index]   (0)

In [121]:
mdf[1.0].hvplot.scatter() * s.hvplot.scatter()

:Overlay
   .Scatter.A_1_full_stop_0 :Scatter   [index]   (1.0)
   .Scatter.A_0             :Scatter   [index]   (0)

In [37]:
mdf = modeler.mdf
mdf

,1.0,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,...,2.6,2.7,2.8,2.9,3.0,3.1,3.2,3.3,3.4,3.5
1970-01-01 07:37:30,107.270191,107.528061,107.779415,108.023947,108.369778,108.598592,108.820973,109.039640,109.252966,109.462352,...,110.970441,111.146759,111.319293,111.488895,111.656381,111.820842,112.051587,112.209273,112.365295,112.519002
1970-01-01 07:52:30,106.014645,106.315742,106.509278,106.794494,107.070521,107.339026,107.598814,107.768526,108.017348,108.260454,...,109.740353,109.879612,110.080520,110.277351,110.471025,110.660548,110.788612,110.972131,111.153081,111.330785
1970-01-01 08:07:30,104.898919,105.234257,105.378330,105.695885,106.002547,106.219531,106.508907,106.712285,106.913670,107.183694,...,108.642827,108.808860,108.973611,109.192133,109.406771,109.562317,109.714717,109.917837,110.066833,110.263647
1970-01-01 08:22:30,103.889654,104.169726,104.439633,104.703502,104.959757,105.207491,105.450025,105.756698,105.916917,106.208861,...,107.648618,107.840177,108.025791,108.209629,108.441361,108.567141,108.742470,108.961402,109.082287,109.294728
1970-01-01 08:37:30,102.974047,103.206056,103.582387,103.799232,104.011513,104.288961,104.555374,104.817506,105.008738,105.259013,...,106.743543,106.953242,107.159112,107.312295,107.511863,107.705660,107.852371,108.041134,108.226228,108.409148
1970-01-01 08:52:30,102.060631,102.396062,102.722340,102.969321,103.209728,103.445140,103.735096,103.957991,104.175226,104.388794,...,105.910869,106.138287,106.314153,106.487215,106.658439,106.869133,107.034435,107.236581,107.436833,107.593504
1970-01-01 09:07:30,101.290278,101.648656,101.930508,102.203050,102.467444,102.668470,102.919993,103.166277,103.405566,103.640136,...,105.142305,105.340457,105.533980,105.723961,105.911434,106.094909,106.276095,106.491588,106.666182,106.837890
1970-01-01 09:22:30,100.577466,100.892281,101.197466,101.491891,101.722181,101.946910,102.218106,102.433435,102.691901,102.944665,...,104.427255,104.640376,104.848251,105.014302,105.215777,105.412693,105.571346,105.761750,105.949769,106.134425
1970-01-01 09:37:30,99.911988,100.190415,100.515633,100.776310,101.028530,101.276717,101.565211,101.798597,102.026277,102.294199,...,103.798499,103.986972,104.207295,104.351575,104.565207,104.739875,104.911954,105.081181,105.280817,105.476709
1970-01-01 09:52:30,99.290865,99.588566,99.878982,100.109811,100.431317,100.648985,100.906719,101.202820,101.402753,101.684235,...,103.171900,103.373261,103.570529,103.730298,103.954554,104.108903,104.294097,104.474601,104.653756,104.859462


In [47]:
all(ds.compute() == ds2.compute())

True

In [7]:
import hvplot.xarray

In [9]:
stacked.sel(chi=[1, 2.4, 3], method="nearest").hvplot(x="time", by="chi").opts()

:DynamicMap   [lat]
   :NdOverlay   [chi]
      :Curve   [time]   (cooling_curves)

In [8]:
stacked.sel(lat=[11, 15, 18, 20]).hvplot(x="time", by="lat").opts()

KeyError: "not all values found in index 'lat'"